In [3]:
import warnings
warnings.filterwarnings('ignore')

from groq import Groq
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
client = Groq()

In [4]:
def get_completion(prompt, model="llama3-70b-8192"):
    messages = [{
    "role":"user", 
    "content":prompt
    }]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content


In [5]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [6]:
prompt = f"""
    what is the overall sentimant of the following product review \
    which is delimited with triple backticks?
    
    Review text:```{lamp_review}```
"""
response = get_completion(prompt)
print(response)


The overall sentiment of this product review is **POSITIVE**. The reviewer is very satisfied with their purchase, mentioning several positive aspects such as the product's features, fast shipping, and excellent customer service. They also express appreciation for the company's responsiveness and care for their customers. The tone is enthusiastic and praising, with no negative comments or complaints.


In [7]:
prompt = f"""
    what is the overall sentimant of the following product review \
    which is delimited with triple backticks?
    
    Give your answer in single word either "Negetive" or "Positive
    Review text:```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

Positive


In [8]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Here is the list of emotions expressed by the writer of the review:

satisfaction, gratitude, happiness, trust, admiration


In [9]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

No



#### Extract product and compny name from customer review

In [10]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Here is the response in JSON format:

`{ "Item": "lamp", "Brand": "Lumina" }`


#### Doing multitask at once

In [12]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
"Sentiment": "positive",
"Anger": false,
"Item": "lamp",
"Brand": "Lumina"
}


str

In [14]:
#### Inferring topics

In [15]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

In [26]:
prompt = f"""
Determine five topic that are being discussed \
in the following text, which is delimited by triple backticks.

Make each item one or two words long
Format your response as a list of topics seperated by commas.
text sample: ```{story}```

Do not include any introductory text or explanations.   
"""
response = get_completion(prompt)
print(response.split(','))

['Job Satisfaction', ' NASA', ' Government', ' Survey', ' Employees']


#### Make a news alert for certain topics

In [30]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [34]:
prompt = f"""
Determine weather each item in the following list of \
topics is a topic in the text given below, which is delimited with triple
backticks.

Give your answer in 0 or 1 for each topic. \

List of topics: {", ".join(topic_list)}

Text_sample:```{story}```

Do not include any introductory text or explanations. 
"""

response = get_completion(prompt)
print(response)

Here are the answers:

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [39]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')[2:]}
if topic_dict['nasa'] == 1:
    print("ALERT: NEW NASA STORY")

ALERT: NEW NASA STORY
